# TotalSegmentator

### Robust segmentation of 104 anatomic structures in ct images

Kindly install the TotalSegmentator package following the steps described in: https://github.com/wasserth/TotalSegmentator

In [1]:
"""
KB Girum
Script to run the totalsegmentator batchwise 
"""

# import library
import os
import glob
from pathlib import Path

import numpy as np
from tqdm import tqdm

import time 


In [2]:
# Paralle computing 
# import ray 
# ray.init(log_to_driver=False, num_cpus=os.cpu_count())
# @ray.remote(num_cpus=os.cpu_count())
# def worker(path_id):
#     !TotalSegmentator  -i  {path_id} -o segmentations --ml 

### 1.  Prepare the data (CT in .nii.gz format) as in the following file structure

MainFolder <br>
------patient1  <br>
---------CT  <br>
------------name.nii.gz  <br>

------patient2  <br>
---------CT  <br>
------------name.nii.gz  <br>
.<br>
.<br>
.<br>
------patientN  <br>
---------CT  <br>
------------name.nii.gz  <br>

The folder names such as MainFolder, patient1, patient2 .. patientN, name.nii.gz could be any name.

### 2. For each patient id, segmented anatomical structures will be saved under subfoder "organ_segmentation"

In [3]:
# Provide the path to the main folder consistes of data under the main folder name (e.g., MainFolder) with
# subfolder patient ids and CT subfolder as described above. 

# path to the data (e.g., with main folder name "MainFolder")
data_path = r"....givethepath../MainFolder/"

# get list of all patient ids 
all_cases = os.listdir(data_path)
# save cases not segmented by TotalSegmentator. 
# The reason could be not readable .nii.gz file or empty folder 
cases_not_segmented = []

# to get the computating time
start_time = time.time()

# read all cases 
for id in tqdm(all_cases):
    path_id = os.path.join(data_path, str(id), "CT")
    path_id_ct = [ id_here for id_here in glob.glob(path_id + '/*.nii.gz')]
    if not len(path_id_ct):
        print(f" {id} file not found {path_id_ct}")
    else: 
        print(path_id)
        path_id_ct = path_id_ct[0]
        
        # Create a subfolder named "organ_segmentation" beside the subfolder "CT"
        path_save = os.path.join(data_path, str(id), "organ_segmentation")
        if not os.path.isdir(path_save):
            os.mkdir(path_save) 
        else:
            print(f"patient {id} already segmented before")
#             continue # uncomment not to segment it again if already segmented 

        print(f'segmentation will be saved in: {path_save}')
        os.chdir(path_save)
        try:
            !TotalSegmentator  -i  {path_id_ct} -o segmentations --fast #--roi_subset
        except:
            cases_not_segmented.append(id)
            pass 
print('== total computing time in seconds ==', (time.time()-start_time))
print(f"list of cases not segmented {cases_not_segmented}")
